# 事前準備

## Google Drive

Google Driveをマウント

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 地域コード

GoogleDriveから地域一覧を取得する

In [ ]:
import json

with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)

print(arealist)

# DataFrameに格納する関数

estat-APIのレスポンスJSONをDataFrameに格納する関数

In [19]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

import urllib.parse
import urllib.request
import json
import pandas as pd

"""
estat-APIのレスポンスを整形してDataFrameに変換
"""

def get_estat_dataframe(params):
  res = get_estat_response(params)
  df_res = conv_estat_response_to_dataframe(res)
  df_res = format_estat_dataframe(df_res)

  return df_res

"""
estat-APIのレスポンスを取得する関数
"""

def get_estat_response(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))


"""
estat-APIのレスポンスをDataFrameに変換する
"""

def conv_estat_response_to_dataframe(response):

  # VALUEをDataFrameに変換
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  df = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df = pd.merge(df, df_class, on=key_name, how='outer')

  # 統計情報を追加
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']
  df['statsDataId'] = TABLE_INF['@id']
  df['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df

"""
DataFrameを整形する
"""

def  format_estat_dataframe(df_arg):

  # 列を抽出して名前変更
  df_res =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','$','@unit']]
  df_res = df_res.rename(columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','$':'value','@unit':'unit'})

  # 欠損データ等処理
  df_res = df_res.dropna()
  df_res = df_res.replace('-', '0')
  df_res = df_res.replace('X', '0')

  # categoryNameから不要な情報（categoryCode）を削除
  df_res['categoryName'] = df_res.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df_res['timeCode'] = df_res.apply(lambda x: x['timeCode'][:4], 1)
  
  # 地域情報をマージ
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    df_arealist = pd.json_normalize(arealist)  
  df_res = pd.merge(df_res, df_arealist, on='areaCode', how='left')

  return df_res

## 市区町村テスト

In [6]:
params = {
    'statsDataId':'0000020201',
    'cdCat01':'A1101',
    }

In [7]:
df_population_city = get_estat_dataframe(params)
df_population_city

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType
0,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,01100,1401757,人,0.0,1.0,北海道,01100,札幌市,2,札幌市,city
1,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,01101,181806,人,1.0,1.0,北海道,01101,札幌市中央区,1,札幌市中央区,city
2,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,01102,195370,人,2.0,1.0,北海道,01102,札幌市北区,1,札幌市北区,city
3,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,01103,213310,人,3.0,1.0,北海道,01103,札幌市東区,1,札幌市東区,city
4,0000020201,社会・人口統計体系,A1101,総人口,1980,1980年度,01104,228061,人,4.0,1.0,北海道,01104,札幌市白石区,1,札幌市白石区,city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15308,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,47361,7755,人,1917.0,47.0,沖縄県,47361,久米島町,0,久米島町,city
15309,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,47362,29066,人,1918.0,47.0,沖縄県,47362,八重瀬町,0,八重瀬町,city
15310,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,47375,1194,人,1919.0,47.0,沖縄県,47375,多良間村,0,多良間村,city
15311,0000020201,社会・人口統計体系,A1101,総人口,2015,2015年度,47381,3998,人,1920.0,47.0,沖縄県,47381,竹富町,0,竹富町,city


## 都道府県テスト

In [20]:
params = {
    'statsDataId':'0000010103',
    'cdCat01': 'C3120,C312001,C312002,C312003,C31201',
    }

In [21]:
df_population_prefecture = get_estat_dataframe(params)
df_population_prefecture

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType
0,0000010103,社会・人口統計体系,C3120,漁業産出額,1975,1975年度,00000,1773957,百万円,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000010103,社会・人口統計体系,C312001,海面漁業産出額,1975,1975年度,00000,1520345,百万円,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000010103,社会・人口統計体系,C312002,海面養殖業産出額,1975,1975年度,00000,253612,百万円,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000010103,社会・人口統計体系,C3120,漁業産出額,1975,1975年度,01000,277527,百万円,0.0,1.0,北海道,None,None,None,北海道,prefecture
4,0000010103,社会・人口統計体系,C312001,海面漁業産出額,1975,1975年度,01000,266132,百万円,0.0,1.0,北海道,None,None,None,北海道,prefecture
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,0000010103,社会・人口統計体系,C312003,海面養殖業産出額（種苗養殖を除く）,2019,2019年度,46000,54866,百万円,45.0,46.0,鹿児島県,None,None,None,鹿児島県,prefecture
6476,0000010103,社会・人口統計体系,C31201,海面漁業・養殖業産出額,2019,2019年度,46000,76048,百万円,45.0,46.0,鹿児島県,None,None,None,鹿児島県,prefecture
6477,0000010103,社会・人口統計体系,C312001,海面漁業産出額,2019,2019年度,47000,11900,百万円,46.0,47.0,沖縄県,None,None,None,沖縄県,prefecture
6478,0000010103,社会・人口統計体系,C312003,海面養殖業産出額（種苗養殖を除く）,2019,2019年度,47000,9147,百万円,46.0,47.0,沖縄県,None,None,None,沖縄県,prefecture


# BigQueryに保存

In [ ]:
!pip install pandas-gbq

In [10]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# BigQueryのデータセット定義
project_id='primal-buttress-342908'
dataset_id = 'test'

def save_gbq_test(df,table_id):
  client = bigquery.Client()

  try:
    client.get_table(f'{project_id}.{dataset_id}.{table_id}')  
    print("Table {} already exists.".format(table_id))

  except NotFound:
    df.to_gbq(f'{dataset_id}.{table_id}', project_id=project_id, if_exists="replace")
    print(f'{table_id}を保存しました')

In [13]:
df = get_estat_dataframe(params)
save_gbq_test(df,'total-population-prefecture')

RefreshError: ignored